In [1]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 32.4 MB/s eta 0:00:00


In [52]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [72]:
from openai import OpenAI
import os

client = OpenAI(
    # This is the default and can be omitted
    api_key="",
)

def query_gpt4(prompt, input, temperature=0.2):
    """
    Sends a prompt to GPT-4 and retrieves the response with a specified temperature.

    Parameters:
        prompt (str): The input prompt for GPT-4.
        temperature (float): Controls the randomness in the response, lower is more deterministic.

    Returns:
        str: The response from GPT-4.
    """

    try:
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": prompt,
                },
                {
                    "role": "user",
                    "content": input
                }
            ],
            model="gpt-4-turbo",
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [2]:
import fitz

In [3]:
pdf_path = "DDIA.pdf"

In [4]:
doc = fitz.open(pdf_path)

In [41]:
start_page, end_page = 8, 13

In [42]:
pages = [doc.load_page(i) for i in range(start_page, end_page + 1)]

In [49]:
raw_texts = '\n'.join([str(page.get_text()) for page in pages])

In [50]:
raw_texts

'Table of Contents\nPreface. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  xiii\nPart I. \nFoundations of Data Systems\n1. Reliable, Scalable, and Maintainable Applications. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  3\nThinking About Data Systems                                                                                       4\nReliability                                                                                                                          6\nHardware Faults                                                                                                           7\nSoftware Errors                                                                                                             8\nHuman Errors                                                                                                               9\nHow Important Is Reliability?         

In [57]:
bookSchema = """
type: object
properties:
  id:
    type: string
    description: The ID of the textbook.
  title:
    type: string
    description: The title of the textbook.
  chapters:
    type: array
    description: A list of chapters in the textbook.
    items:
      id:
        type: string
        description: The ID of the chapter.
  metadata:
    type: object
    additionalProperties: true
    description: Metadata for the textbook.

required:
  - id
  - title
  - chapters"""

chapterSchema = """
type: object
properties:
  id:
    type: string
    description: The ID of the chapter.
  title:
    type: string
    description: The title of the chapter.
  range:
    type: array
    items:
      type: integer
    minItems: 2
    maxItems: 2
    description: The start and end pages of the chapter.
  subchapters:
    type: array
    items:
      id:
        type: string
        description: The ID of the subchapter.
    description: A list of subchapters.
  metadata:
    type: object
    additionalProperties: true
    description: Metadata for the chapter.

required:
  - id
  - title
  - range"""

In [70]:
cleaning_prompt_template = f"""
You are given as input the table of contents of Designing Data-Intensive Applications. The text is scraped from a PDF file, so there are headers/footers like "table of contents" you have to ignore.
Help me to clean this table of contents into a list of major chapters and subchapters, which may have subchapters of their own, each pointing to a page number.
Chapters that aren't technical content like "Preface" or "Glossary" should be omitted.

Clean it into the following YAML schema:
Book shema:
{bookSchema}

Chapter schema:
{chapterSchema}

Don't worry about generating the IDs for now.
"""

In [83]:
precleaning_prompt_template = f"""
You are given as input the table of contents of Designing Data-Intensive Applications. The text is scraped from a PDF file, so there are headers/footers like "table of contents" you have to ignore.
Help me to clean this table of contents into a list of major chapters and subchapters.
Chapters that aren't technical content like "Preface" or "Glossary" should be omitted.

Print into the following example yaml format, always reproducing the page number following the chapter name.

If a chapter starts on page 107, it should read

chapter name 107

Example:

- Major chapter name 1
  - subchapter name 1
    - subsubchapter name 3
    - subsubchapter name 4
  - subchapter name 6
  - subchapter name 7
- Major chapter name 9
  - subchapter name 10
  - subchapter name 12"""

In [75]:
test = query_gpt4("hi", "hi there")
print(test)

Hello! How can I help you today?


In [84]:
cleaned = query_gpt4(precleaning_prompt_template, raw_texts)

In [85]:
cleaned

'```yaml\n- Foundations of Data Systems 3\n  - Reliable, Scalable, and Maintainable Applications 3\n    - Thinking About Data Systems 4\n    - Reliability 6\n    - Hardware Faults 7\n    - Software Errors 8\n    - Human Errors 9\n    - How Important Is Reliability? 10\n    - Scalability 10\n    - Describing Load 11\n    - Describing Performance 13\n    - Approaches for Coping with Load 17\n    - Maintainability 18\n    - Operability: Making Life Easy for Operations 19\n    - Simplicity: Managing Complexity 20\n    - Evolvability: Making Change Easy 21\n  - Summary 22\n  - Data Models and Query Languages 27\n    - Relational Model Versus Document Model 28\n    - The Birth of NoSQL 29\n    - The Object-Relational Mismatch 29\n    - Many-to-One and Many-to-Many Relationships 33\n    - Are Document Databases Repeating History? 36\n    - Relational Versus Document Databases Today 38\n    - Query Languages for Data 42\n    - Declarative Queries on the Web 44\n    - MapReduce Querying 46\n   

In [86]:
cleaned = cleaned.replace("```yaml\n", "")
cleaned = cleaned.replace("```", "")

'- Foundations of Data Systems 3\n  - Reliable, Scalable, and Maintainable Applications 3\n    - Thinking About Data Systems 4\n    - Reliability 6\n    - Hardware Faults 7\n    - Software Errors 8\n    - Human Errors 9\n    - How Important Is Reliability? 10\n    - Scalability 10\n    - Describing Load 11\n    - Describing Performance 13\n    - Approaches for Coping with Load 17\n    - Maintainability 18\n    - Operability: Making Life Easy for Operations 19\n    - Simplicity: Managing Complexity 20\n    - Evolvability: Making Change Easy 21\n  - Summary 22\n  - Data Models and Query Languages 27\n    - Relational Model Versus Document Model 28\n    - The Birth of NoSQL 29\n    - The Object-Relational Mismatch 29\n    - Many-to-One and Many-to-Many Relationships 33\n    - Are Document Databases Repeating History? 36\n    - Relational Versus Document Databases Today 38\n    - Query Languages for Data 42\n    - Declarative Queries on the Web 44\n    - MapReduce Querying 46\n    - Graph-

In [108]:
class Chapter:
    def __init__(self, title, firstPage, subchapters):
        self.id = id
        self.title = title
        self.firstPage = firstPage
        self.subchapters = subchapters

    def setLastPage(self, lastPage):
        self.lastPage = lastPage
        self.range = [self.firstPage, lastPage]



In [110]:
def parse(str_list, index = 0):
    """
    Input: list of strings in a yaml-like format
    Output: (list of Chapter objects, index of last chapter)
    """
    def parse_chapter_info(string):
        """
        returns (title, firstPage, indentation)
        """
        # print("parsing", string)
        indentation = 0
        while string[indentation] == " ":
            indentation += 1

        number_start = len(string) - 1
        while string[number_start-1].isnumeric():
            number_start -= 1
        title = string[indentation+2:number_start].strip()
        # print(string, number_start, string[number_start:].strip())
        firstPage = int(string[number_start:].strip())
        return title, firstPage, indentation

    chapters = []
    while index < len(str_list):
        string = str_list[index]
        title, firstPage, indentation = parse_chapter_info(string)
        index += 1
        if index < len(str_list) and parse_chapter_info(str_list[index])[2] > indentation:
            subchapters, index = parse(str_list, index)
        else:
            subchapters = []
        chapters.append(Chapter(title, firstPage, subchapters))
        if index < len(str_list) and parse_chapter_info(str_list[index])[2] < indentation:
            break
    return chapters, index



cleanedList = list(filter(lambda x: x != "", cleaned.split('\n')))
chapters, index = parse(cleanedList)
print(chapters)

[<__main__.Chapter object at 0x7b4ee43758a0>, <__main__.Chapter object at 0x7b4ee4133160>, <__main__.Chapter object at 0x7b4ee4133c40>]


In [111]:
for chapter in chapters:
    print(chapter.title, chapter.firstPage)

Foundations of Data Systems 3
Distributed Data 151
Derived Data 389


In [112]:
def setPageRanges(chapters, lastPage=543):
    for idx, chapter in reversed(list(enumerate(chapters))):
        chapter.setLastPage(lastPage)
        lastPage = chapters[idx].firstPage
        setPageRanges(chapter.subchapters, lastPage)

setPageRanges(chapters)

In [115]:
for chapter in chapters:
    print(chapter.title, chapter.range)
for subchapter in chapters[0].subchapters:
    print(subchapter.title, subchapter.range)

Foundations of Data Systems [3, 151]
Distributed Data [151, 389]
Derived Data [389, 543]
Reliable, Scalable, and Maintainable Applications [3, 22]
Summary [22, 27]
Data Models and Query Languages [27, 63]
Summary [63, 69]
Storage and Retrieval [69, 103]
Summary [103, 111]
Encoding and Evolution [111, 139]
Summary [139, 3]


# Print the parsed AST to YAML!!!

In [118]:
!pip install shortuuid

In [119]:
import shortuuid
import yaml

In [124]:
def random_uuid():
    return shortuuid.uuid()

def chapter_id():
    return "chapter-" + random_uuid()

def book_id():
    return "book-" + random_uuid()

def subchapter_id(subchapter):
    return random_uuid()

book_id()

'book-ZatxnaHzKrXJWec7TqPzve'

In [125]:
bookDict = {
    "id": book_id(),
    "title": "Designing Data-Intensive Applications",
    "chapters": [],
    "metadata": {
        "authors": ["Martin Kleppmann"],
        "publisher": "Springer",
        "date": "2019",
        "stars": 1
    }
}


In [126]:
def getChapterDicts(chapters, bookFrom, depth=1):
    chapterDicts = []
    for chapter in chapters:
        subChapterDicts = getChapterDicts(chapter.subchapters, bookFrom, depth+1)
        chapterDict = {
            "id": chapter_id(),
            "title": chapter.title,
            "range": chapter.range,
            "subchapters": [subDict["id"] for subDict in subChapterDicts],
            "metadata": {
                "book": bookFrom,
                "depth": depth
            }
        }
        chapterDicts.append(chapterDict)
        chapterDicts.extend(subChapterDicts)
    return chapterDicts

chapterDicts = getChapterDicts(chapters, book_id())

In [130]:
bookDict["chapters"] = [chapterDict["id"] for chapterDict in chapterDicts if chapterDict["metadata"]["depth"] == 1]

In [131]:
print(yaml.dump(bookDict))

chapters:
- chapter-XKbs6ijJhgsWkZnJyxNrCS
- chapter-4wHZ6nLjpStEPaooJWxhto
- chapter-PLWncfnaFb62kDhtgN9U4A
id: book-D2c96cyaQuV6cvL2nhgt4t
metadata:
  authors:
  - Martin Kleppmann
  date: '2019'
  publisher: Springer
  stars: 1
title: Designing Data-Intensive Applications



In [132]:
print(yaml.dump(chapterDicts))

- id: chapter-XKbs6ijJhgsWkZnJyxNrCS
  metadata:
    book: book-8Q7HM3FUviMzFCjzD4fQat
    depth: 1
  range:
  - 3
  - 151
  subchapters:
  - chapter-EJYyCecRmRDSyCiRwDazaH
  - chapter-cve24umMsFh6w9pC39Hvvt
  - chapter-NnwnZFkmi8E7dZ8GRrRpW6
  - chapter-HsWu6Bw4HCP7s9wYWZA8rq
  - chapter-HagzfLmF3San57r6KZrXNc
  - chapter-W2TbcMfhqBqiGgJstV4QVn
  - chapter-oAS7CWjBKR8y4QgUTs65Cj
  - chapter-hiAhyjUAQFzSfnhTw58c9w
  - chapter-HUUNb5RsAHK9uCteTYuKea
  - chapter-7k7y8BxgrPaEm4UobU7ASb
  - chapter-ixMMYryNDQJrq6mA9XNhyN
  - chapter-NtRUUBQYJsgCvqmmTpJ5JQ
  - chapter-3ndt4NGd7tXfpwFNpH6WiN
  - chapter-Y65migRpGjd74PhiHe6HPh
  - chapter-6SY4J9a9UjUeP2TQHrdRFN
  - chapter-GXgy5m5Ccakpiz68soZhQS
  - chapter-Hoe7Ryy2MNaKh3ptnpv9zt
  - chapter-cXCkCxJAoJRihczND5BfFv
  - chapter-mxoCBgBYibrKTdm4GJsV7s
  - chapter-RQNpk6xmduGYmtjRQd3ZMY
  - chapter-3p7TeLMh2FNgzTsZg3YMAR
  - chapter-DH5qtsbjY8ZFx3hwvmgSey
  - chapter-QrvywwLxT9tUcZpfwpMqrR
  - chapter-BWdo6yo8mC9R5vKjZ4uUGB
  - chapter-A38DnUA7rH